# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andrewnachtigal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andrewnachtigal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/andrewnachtigal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# import libraries
import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier


In [4]:
# load data from database
database_filepath = "../data/disaster_response.db"
engine = create_engine('sqlite:///' + database_filepath)
df = pd.read_sql_table('message_table', engine)


In [5]:
# drop child alone - has all zeros
df = df.drop(['child_alone'],axis=1)


In [8]:
# Extract X and y variables from the data for modelling
X = df['message']
y = df.iloc[:,4:]


In [26]:
# category names
category_names = list(y.columns.values)
category_names


['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [ ]:
# message data visualization



### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    """Tokenize the text function
    
    Args:
        text: Text message which needs to be tokenized
    Returns:
        clean_tokens: List of tokens extracted from the provided text
    """
    
    # Replace urls with urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract urls from text & replace url with url placeholder string
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Extract word tokens from text
    tokens = nltk.word_tokenize(text)
    
    # Lemmanitizer to reduce inflectional forms and derivationally related forms of words.
    lemmatizer = nltk.WordNetLemmatizer()

    # List clean tokens
    #clean_tokens = []
    #for tok in tokens:
    #    clean_tok = lemmatizer.lemmatize(tok).lower().strip()
    #    clean_tokens.append(clean_tok)
    
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    
    return clean_tokens


In [10]:
# from example
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
# from github
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [12]:
# generate all parameters for pipeline
sorted(pipeline.get_params().keys())


['classifier',
 'classifier__estimator',
 'classifier__estimator__algorithm',
 'classifier__estimator__base_estimator',
 'classifier__estimator__learning_rate',
 'classifier__estimator__n_estimators',
 'classifier__estimator__random_state',
 'classifier__n_jobs',
 'features',
 'features__n_jobs',
 'features__text_pipeline',
 'features__text_pipeline__count_vectorizer',
 'features__text_pipeline__count_vectorizer__analyzer',
 'features__text_pipeline__count_vectorizer__binary',
 'features__text_pipeline__count_vectorizer__decode_error',
 'features__text_pipeline__count_vectorizer__dtype',
 'features__text_pipeline__count_vectorizer__encoding',
 'features__text_pipeline__count_vectorizer__input',
 'features__text_pipeline__count_vectorizer__lowercase',
 'features__text_pipeline__count_vectorizer__max_df',
 'features__text_pipeline__count_vectorizer__max_features',
 'features__text_pipeline__count_vectorizer__min_df',
 'features__text_pipeline__count_vectorizer__ngram_range',
 'features__

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
# pipeline - from example
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline.fit(X_train, y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
# initial model
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# test data classification report
print(classification_report(y_test.values, y_prediction_test, target_names = y.columns.values))


                        precision    recall  f1-score   support

               related       0.82      0.94      0.88      4987
               request       0.76      0.55      0.64      1123
                 offer       0.00      0.00      0.00        29
           aid_related       0.77      0.61      0.68      2706
          medical_help       0.65      0.26      0.37       551
      medical_products       0.61      0.30      0.41       338
     search_and_rescue       0.46      0.14      0.21       164
              security       0.17      0.03      0.05       100
              military       0.54      0.25      0.35       209
                 water       0.75      0.64      0.69       408
                  food       0.81      0.68      0.74       733
               shelter       0.77      0.54      0.64       593
              clothing       0.70      0.43      0.54       104
                 money       0.58      0.25      0.35       156
        missing_people       0.44      

/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
# training data classification report
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))



                         precision    recall  f1-score   support

               related       0.83      0.95      0.88     15107
               request       0.79      0.55      0.65      3351
                 offer       0.29      0.09      0.14        89
           aid_related       0.77      0.60      0.68      8154
          medical_help       0.66      0.29      0.40      1533
      medical_products       0.71      0.38      0.49       975
     search_and_rescue       0.67      0.24      0.35       560
              security       0.45      0.08      0.14       371
              military       0.68      0.37      0.48       651
                 water       0.77      0.67      0.72      1264
                  food       0.82      0.69      0.75      2190
               shelter       0.79      0.54      0.64      1721
              clothing       0.76      0.47      0.58       301
                 money       0.64      0.32      0.43       448
        missing_people       0.74    

/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
# grid search parameters
parameters = {
        'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
        'classifier__estimator__n_estimators': [10, 20, 40]
    }

cv = GridSearchCV(pipeline, param_grid = parameters)
    

In [19]:
# grid search cross validation
cv.fit(X_train, y_train)


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7fe504776a60>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05],
                         'classifier__estimator__n_estimators': [

In [20]:
# grid search cross validator prediction values
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# test data classification report
print(classification_report(y_test.values, y_prediction_test, target_names = y.columns.values))


                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4987
               request       0.89      0.22      0.35      1123
                 offer       0.00      0.00      0.00        29
           aid_related       0.80      0.26      0.39      2706
          medical_help       0.88      0.01      0.03       551
      medical_products       1.00      0.01      0.01       338
     search_and_rescue       1.00      0.01      0.01       164
              security       0.00      0.00      0.00       100
              military       0.00      0.00      0.00       209
                 water       0.61      0.74      0.67       408
                  food       0.78      0.67      0.72       733
               shelter       0.84      0.30      0.44       593
              clothing       1.00      0.01      0.02       104
                 money       0.00      0.00      0.00       156
        missing_people       0.00      

/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# training data classification report
print('\n',classification_report(y_train.values, y_prediction_train, target_names = y.columns.values))



                         precision    recall  f1-score   support

               related       0.77      1.00      0.87     15107
               request       0.89      0.22      0.35      3351
                 offer       0.00      0.00      0.00        89
           aid_related       0.80      0.26      0.39      8154
          medical_help       0.70      0.01      0.02      1533
      medical_products       0.60      0.00      0.01       975
     search_and_rescue       0.82      0.02      0.03       560
              security       0.00      0.00      0.00       371
              military       0.00      0.00      0.00       651
                 water       0.66      0.74      0.69      1264
                  food       0.77      0.68      0.72      2190
               shelter       0.84      0.31      0.45      1721
              clothing       0.50      0.00      0.01       301
                 money       1.00      0.00      0.01       448
        missing_people       1.00    

/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrewnachtigal/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [23]:
import pickle


In [24]:
# create iterator object 
with open('classifier_0.pkl', 'wb') as f:
    pickle.dump(cv, f)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.